In [2]:
import nest_asyncio
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
import asyncio
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

nest_asyncio.apply()

TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")

# Load TinyLlama model and tokenizer
try:
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Replace with the exact TinyLlama model path
    logger.info(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Set the padding token (TinyLlama models may not define a pad token by default)
    tokenizer.pad_token = tokenizer.eos_token

    logger.info("Model and tokenizer loaded successfully.")
except Exception as e:
    logger.error(f"Error loading model or tokenizer: {e}")
    raise e

def process_with_tinyllama(prompt):
    """
    Processes user input with TinyLlama and generates a detailed response.
    """
    try:
        # Clean and identify the prompt
        topic = prompt.lower().replace("tell me about", "").strip()
        enhanced_prompt = f"Write a detailed and informative explanation about {topic.capitalize()}, including examples and interesting facts."

        # Tokenize input and create attention mask
        inputs = tokenizer(enhanced_prompt, return_tensors="pt", padding=True, truncation=True)
        inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).long()

        # Generate output
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=300,
            temperature=1.2,
            top_p=0.95,
            top_k=50,
            pad_token_id=tokenizer.pad_token_id,
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # Retry with a simplified prompt if the response is unsatisfactory
        if not response.strip() or response.lower() == enhanced_prompt.lower():
            logger.warning("Retrying with a simplified prompt...")
            fallback_prompt = f"Tell me all you know about {topic.capitalize()}."
            inputs = tokenizer(fallback_prompt, return_tensors="pt", padding=True, truncation=True)
            inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).long()
            outputs = model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=300,
                temperature=1.0,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
            )
            response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # Log details for debugging
        logger.info(f"Constructed Prompt: {enhanced_prompt}")
        logger.info(f"Model Output: {response}")

        return response
    except Exception as e:
        logger.error(f"Error during TinyLlama processing: {e}")
        return "Sorry, an error occurred while processing your request."

async def start(update: Update, context: CallbackContext):
    """
    Handles the /start command.
    """
    await update.message.reply_text("Hello! I'm DSSS AI assistant powered by TinyLlama. Ask me anything!")

async def echo(update: Update, context: CallbackContext):
    """
    Handles user input and generates a response using TinyLlama.
    """
    user_message = update.message.text
    try:
        logger.info(f"Received message: {user_message}")
        response = await asyncio.to_thread(process_with_tinyllama, user_message)
        logger.info(f"Generated response: {response}")
        await update.message.reply_text(response)
    except Exception as e:
        logger.error(f"Error in message handling: {e}")
        await update.message.reply_text("Sorry, I couldn't process your request.")

def run_bot():
    """
    Creates and runs the Telegram bot application in a Jupyter-friendly way.
    """
    application = Application.builder().token(TELEGRAM_TOKEN).build()

    # Command and message handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))

    logger.info("Bot is starting...")
    print("Bot is running. Stop the kernel to terminate.")

    try:
        # For environments like Jupyter, use the currently running event loop
        loop = asyncio.get_event_loop()

        if loop.is_running():
            # Use `run_polling` with asyncio.create_task for running event loops
            logger.info("Detected a running event loop. Using create_task.")
            loop.create_task(application.run_polling())
        else:
            # If no loop is running, use asyncio.run
            asyncio.run(application.run_polling())
    except RuntimeError as e:
        logger.error(f"Runtime error occurred: {e}")
    except KeyboardInterrupt:
        logger.info("Bot stopped by user.")


# Run the bot
run_bot()


2025-01-25 16:49:19,482 - __main__ - INFO - Loading model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
2025-01-25 16:49:38,311 - __main__ - INFO - Model and tokenizer loaded successfully.
2025-01-25 16:49:40,171 - __main__ - INFO - Bot is starting...
2025-01-25 16:49:40,173 - __main__ - INFO - Detected a running event loop. Using create_task.


Bot is running. Stop the kernel to terminate.


2025-01-25 16:49:40,527 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7027664349:AAGle-6J7OonsOjCrG7aRaH4aBADDT1btBE/getMe "HTTP/1.1 200 OK"
2025-01-25 16:49:40,571 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7027664349:AAGle-6J7OonsOjCrG7aRaH4aBADDT1btBE/deleteWebhook "HTTP/1.1 200 OK"
2025-01-25 16:49:40,575 - telegram.ext.Application - INFO - Application started
2025-01-25 16:49:43,301 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7027664349:AAGle-6J7OonsOjCrG7aRaH4aBADDT1btBE/getUpdates "HTTP/1.1 200 OK"
2025-01-25 16:49:43,441 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7027664349:AAGle-6J7OonsOjCrG7aRaH4aBADDT1btBE/sendMessage "HTTP/1.1 200 OK"
2025-01-25 16:49:53,371 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7027664349:AAGle-6J7OonsOjCrG7aRaH4aBADDT1btBE/getUpdates "HTTP/1.1 200 OK"
2025-01-25 16:50:01,110 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7027664349:AA